# Sensibilidad OIS

**Definición:** una curva cupón cero es una colección ordenada de plazos y tasas de interés donde cada una de las tasas es una  tasa adecuada para traer a valor presente un flujo de caja al plazo que corresponde a la tasa.

Consideremos un instrumento financiero cuyo valor depende de una curva cupón cero. Por ejemplo, un swap de tasa de interés. Matemáticamente, esta dependencia se expresa como:

$$
V = V\left(z_1,\ldots,z_n;\alpha\right)
$$

donde $z_1,\ldots,z_N$ son los valores de las tasas de la curva y $\alpha$ representa un vector de parámetros adicionales (como la tasa de cupón en un bono o un tipo de cambio en un swap de monedas).

**Definición:** la cantidad $\Delta_i$ de $V$ respecto a $z_i$ se define como:

$$
\Delta_i=\frac{\partial V}{\partial z_i}
$$

**Definición:** la sensibilidad de $V$ respecto a un cambio $\delta z_i$ (positivo o negativo) en la tasa $z_i$ se define como:

$$
S\left(V,z_i,\delta_i\right)=V\left(z_1,...,z_i+\delta_i,\ldots,z_n\right)-V\left(z_1,\ldots,z_i,\ldots,z_n\right)
$$

Cuando se busca conocer la sensibilidad a un cambio $\delta z_i$  en la tasa $z_i$, donde $\delta z_i$ representa una magnitud sin signo, se utiliza la siguiente definición:

**Definición:** la sensibilidad de $V$ respecto a un cambio de magnitud $\delta z_i$ en la tasa $z_i$ se define como:

$$
\overline{S}\left(V,z_i,\delta_i\right)=\frac{V\left(z_1,\ldots ,z_i+\delta_i,\ldots,z_n\right)-V\left(z_1,\ldots,z_i-\delta_i,...,z_n\right)}{2}
$$

En ocasiones, cuando se busca conocer la sensibilidad a un movimiento de magnitud pequeña, resulta conveniente realizar la siguiente aproximación:

$$
\overline{S}\left(V,z_i,\delta_i\right)\approx\Delta_i \cdot\delta_i
$$

este es el caso, cuando se busca conocer esta sensibilidad para un elevado número de operaciones. Es importante considerar que se puede asegurar que esta aproximación es válida cuando la función $V$ satisface los requerimientos de regularidad necesarios para aplicar el [Teorema de Taylor](https://en.wikipedia.org/wiki/Taylor%27s_theorem).


## Configuración

### Librerías

In [1]:
from finrisk import QC_Financial_3 as Qcf
import modules.auxiliary as aux
from functools import partial
from enum import Enum
import pandas as pd

### Variables Globales

In [2]:
class BusCal(Enum):
    NY = 1
    SCL = 2

In [3]:
def get_cal(code: BusCal) -> Qcf.BusinessCalendar:
    """
    """
    if code == BusCal.NY:
        cal = Qcf.BusinessCalendar(Qcf.QCDate(1, 1, 2020), 20)
        for agno in range(2020, 2071):
            f = Qcf.QCDate(12, 10, agno)
            if f.week_day() == Qcf.WeekDay.SAT:
                cal.add_holiday(Qcf.QCDate(14, 10, agno))
            elif f.week_day() == Qcf.WeekDay.SUN:
                cal.add_holiday(Qcf.QCDate(13, 10, agno))
            elif f.week_day() == Qcf.WeekDay.MON:
                cal.add_holiday(Qcf.QCDate(12, 10, agno))
            elif f.week_day() == Qcf.WeekDay.TUE:
                cal.add_holiday(Qcf.QCDate(11, 10, agno))
            elif f.week_day() == Qcf.WeekDay.WED:
                cal.add_holiday(Qcf.QCDate(10, 10, agno))
            elif f.week_day() == Qcf.WeekDay.THU:
                cal.add_holiday(Qcf.QCDate(9, 10, agno))
            else:
                cal.add_holiday(Qcf.QCDate(8, 10, agno))
        cal.add_holiday(Qcf.QCDate(15, 2, 2021))
        
    return cal

In [4]:
frmt = {
    'tasa': '{:.6%}',
    'df': '{:.6%}',
    'valor_tasa': '{:.4%}',
    'spread': '{:.4%}',
    'nominal': '{:,.0f}',
    'interes': '{:,.0f}',
    'amortizacion': '{:,.0f}',
    'flujo': '{:,.0f}',
}

In [8]:
class TypeOis(Enum):
    SOFR = 1
    ICP = 2

In [9]:
type_ois_template = {
    TypeOis.SOFR: {
        'currency': Qcf.QCUSD(),
        'periodicity': Qcf.Tenor('1Y'),
        'stub_period': Qcf.StubPeriod.SHORTFRONT,
        'settlement_lag': 0,
        'calendar': BusCal.NY,
        'bus_adj_rule': Qcf.BusyAdjRules.MODFOLLOW,
        'amort_is_cashflow': True,
        'fixed_rate': Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    },
    TypeOis.ICP: {
        'currency': Qcf.QCCLP(),
        'periodicity': Qcf.Tenor('6M'),
        'stub_period': Qcf.StubPeriod.SHORTFRONT,
        'settlement_lag': 0,
        'calendar': BusCal.SCL,
        'bus_adj_rule': Qcf.BusyAdjRules.MODFOLLOW,
        'amort_is_cashflow': True,
        'fixed_rate': Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    }
}

## Construye Curva Cero Cupón

Se importa la data de la curva cupón cero que fue construida en el notebook 5.

In [10]:
df_curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [11]:
def get_curve_from_dataframe(yf: Qcf.QCYearFraction, wf: Qcf.QCWealthFactor, df_curva: pd.DataFrame):
    """
    Retorna un objeto Qcf.ZeroCouponCurve.
    """
    plazos = Qcf.long_vec()
    tasas = Qcf.double_vec()
    for row in df_curva.itertuples():
        plazos.append(row.plazo)
        tasas.append(row.tasa)
    curva = Qcf.QCCurve(plazos, tasas)
    curva = Qcf.QCLinearInterpolator(curva)
    tipo_tasa = Qcf.QCInterestRate(0.0, yf, wf)
    curva = Qcf.ZeroCouponCurve(curva, tipo_tasa)
    return curva

In [12]:
df_curva.head().style.format(frmt)

,plazo,tasa,df
0,1,0.081111%,99.999778%
1,7,0.084051%,99.998388%
2,14,0.077967%,99.997010%
3,21,0.077358%,99.995549%
4,33,0.078067%,99.992942%


In [13]:
zcc = get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)

Algunos métodos del objeto`zcc`.

In [14]:
plazo = 900
print(f"Tasa a {plazo} días es igual a {zcc.get_rate_at(plazo):.4%}")
print(f"Factor de descuento a {plazo} días es igual a {zcc.get_discount_factor_at(plazo):.6%}")

Tasa a 900 días es igual a 0.0652%
Factor de descuento a 900 días es igual a 99.839384%


## Valorización

In [15]:
def get_ois_using_template(template, type_ois: TypeOis, rp: Qcf.RecPay, notional: float, start_date: Qcf.QCDate,
                           tenor: Qcf.Tenor, fixed_rate_value: float, spread: float, gearing: float):
    """
    """
    template_dict = template[type_ois]
    meses = tenor.get_years() * 12 + tenor.get_months()
    end_date = start_date.add_months(meses)
    template_dict['fixed_rate'].set_value(fixed_rate_value)
    es_bono = False

    # Construye la pata fija
    fixed_rate_leg = Qcf.LegFactory.build_bullet_fixed_rate_leg(
        rp,
        start_date,
        end_date,
        template_dict['bus_adj_rule'],
        template_dict['periodicity'],
        template_dict['stub_period'],
        get_cal(template_dict['calendar']),
        template_dict['settlement_lag'],
        notional,
        template_dict['amort_is_cashflow'],
        template_dict['fixed_rate'],
        template_dict['currency'],
        es_bono)

    # Construye la pata ois
    rp = Qcf.RecPay.PAY if rp == Qcf.RecPay.RECEIVE else Qcf.RecPay.RECEIVE
    icp_clp_leg = Qcf.LegFactory.build_bullet_icp_clp2_leg(
        rp,
        start_date,
        end_date,
        template_dict['bus_adj_rule'],
        template_dict['periodicity'],
        template_dict['stub_period'],
        get_cal(template_dict['calendar']),
        template_dict['settlement_lag'],
        notional,
        template_dict['amort_is_cashflow'],
        spread,
        gearing,
        True
    )

    for i in range(icp_clp_leg.size()):
        cshflw = icp_clp_leg.get_cashflow_at(i)
        cshflw.set_start_date_icp(1.0)
        cshflw.set_end_date_icp(1.0)

    return (fixed_rate_leg, icp_clp_leg)

### Operación Ejemplo

In [17]:
op = get_ois_using_template(
    type_ois_template,
    TypeOis.SOFR,
    Qcf.RecPay.RECEIVE,
    10000000,
    Qcf.QCDate(14, 10, 2020),
    Qcf.Tenor('2Y'),
    .01,
    0.0,
    1.0
)

In [18]:
aux.show_leg(op[0], 'FixedRateCashflow', '').style.format(frmt)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2020-10-14,2021-10-14,2021-10-14,"10,000,000",0,"101,389",True,"101,389",USD,1.0000%,LinAct360
1,2021-10-14,2022-10-14,2022-10-14,"10,000,000","10,000,000","101,389",True,"10,101,389",USD,1.0000%,LinAct360


In [19]:
aux.show_leg(op[1], 'IcpClpCashflow', '').style.format(frmt)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,2020-10-14,2021-10-14,2021-10-14,"-10,000,000",0,True,0,CLP,1.000000,1.000000,0.0000%,-0,0.0000%,1.000000,LinAct360
1,2021-10-14,2022-10-14,2022-10-14,"-10,000,000","-10,000,000",True,"-10,000,000",CLP,1.000000,1.000000,0.0000%,-0,0.0000%,1.000000,LinAct360


#### Valor Presente Pata Fija

In [20]:
vp = Qcf.PresentValue()

In [21]:
fecha_val = Qcf.QCDate(14, 10, 2020)

In [23]:
vp_fija = vp.pv(fecha_val, op[0], zcc)
print(f'El valor presente de la pata fija es: USD {vp_fija:,.2f}')

El valor presente de la pata fija es: USD 10,191,249.71


**Ejercicio:** Replique el valor de la pata fija utilizando los flujos y los factores de descuento que obtiene de la curva `zcc`.

#### Valor Presente Pata Flotante

In [24]:
fwd = Qcf.ForwardRates()

In [25]:
fwd.set_rates_icp_clp_leg(fecha_val, 1.0, op[1], zcc)
vp_flot = vp.pv(fecha_val, op[1], zcc)
print(f'El valor presente de la pata flotante es: USD {vp_flot:,.2f}')

El valor presente de la pata flotante es: USD -10,000,000.00


## Sensibilidad

En `QC_Financial_3` al calcular el valor presente, también se calculan las derivadas del valor presente respecto a cada uno de los vértices de la curva.

### Pata Fija

In [26]:
vp.pv(fecha_val, op[0], zcc)
der = vp.get_derivatives()

Con esas derivadas, se puede calcular la sensibilidad a cada vértice de la curva cupón cero para un movimiento de 1 punto básico.

In [27]:
print(type(der))

<class 'finrisk.QC_Financial_3.double_vec'>


In [28]:
bp = .0001
total = 0
for i, d in enumerate(der):
    total += d * bp
    print(f"Sensibilidad en {i}: {d * bp: 0,.2f}")
print(f"Sensibilidad total: {total:,.2f}")

Sensibilidad en 0:  0.00
Sensibilidad en 1:  0.00
Sensibilidad en 2:  0.00
Sensibilidad en 3:  0.00
Sensibilidad en 4:  0.00
Sensibilidad en 5:  0.00
Sensibilidad en 6:  0.00
Sensibilidad en 7:  0.00
Sensibilidad en 8:  0.00
Sensibilidad en 9:  0.00
Sensibilidad en 10:  0.00
Sensibilidad en 11:  0.00
Sensibilidad en 12:  0.00
Sensibilidad en 13:  0.00
Sensibilidad en 14:  0.00
Sensibilidad en 15: -10.12
Sensibilidad en 16:  0.00
Sensibilidad en 17: -2,016.84
Sensibilidad en 18:  0.00
Sensibilidad en 19:  0.00
Sensibilidad en 20:  0.00
Sensibilidad en 21:  0.00
Sensibilidad en 22:  0.00
Sensibilidad en 23:  0.00
Sensibilidad en 24:  0.00
Sensibilidad en 25:  0.00
Sensibilidad en 26:  0.00
Sensibilidad en 27:  0.00
Sensibilidad en 28:  0.00
Sensibilidad en 29:  0.00
Sensibilidad en 30:  0.00
Sensibilidad en 31:  0.00
Sensibilidad en 32:  0.00
Sensibilidad total: -2,026.97


**Ejercicio:** Verifique por diferencias finitas centrales la sensibilidad en el vértice 17. La aproximación de la derivada por diferencias finitas centrales es:

$$
\begin{equation}
f^{\prime}\left(x\right)\approx\frac{f\left(x+h\right)-f\left(x-h\right)}{2h}
\end{equation}
$$

### Pata Flotante

In [29]:
vp.pv(fecha_val, op[1], zcc)
der = vp.get_derivatives()

In [30]:
bp = .0001
total = 0
for i, d in enumerate(der):
    total += d * bp
    print(f"Sensibilidad en {i}: {d * bp: 0,.2f}")
print(f"Sensibilidad total: {total:,.2f}")

Sensibilidad en 0:  0.00
Sensibilidad en 1:  0.00
Sensibilidad en 2:  0.00
Sensibilidad en 3:  0.00
Sensibilidad en 4:  0.00
Sensibilidad en 5:  0.00
Sensibilidad en 6:  0.00
Sensibilidad en 7:  0.00
Sensibilidad en 8:  0.00
Sensibilidad en 9:  0.00
Sensibilidad en 10:  0.00
Sensibilidad en 11:  0.00
Sensibilidad en 12:  0.00
Sensibilidad en 13:  0.00
Sensibilidad en 14:  0.00
Sensibilidad en 15:  0.70
Sensibilidad en 16:  0.00
Sensibilidad en 17:  1,997.46
Sensibilidad en 18:  0.00
Sensibilidad en 19:  0.00
Sensibilidad en 20:  0.00
Sensibilidad en 21:  0.00
Sensibilidad en 22:  0.00
Sensibilidad en 23:  0.00
Sensibilidad en 24:  0.00
Sensibilidad en 25:  0.00
Sensibilidad en 26:  0.00
Sensibilidad en 27:  0.00
Sensibilidad en 28:  0.00
Sensibilidad en 29:  0.00
Sensibilidad en 30:  0.00
Sensibilidad en 31:  0.00
Sensibilidad en 32:  0.00
Sensibilidad total: 1,998.16


La estructura es la misma que para una pata fija, lo que indica que se debe también incluir la sensibilidad a la curva de proyección.

In [31]:
import numpy as np
bp = .0001
result = []
for i in range(op[1].size()):
    cshflw = op[1].get_cashflow_at(i)
    amt_der = cshflw.get_amount_derivatives()
    df = zcc.get_discount_factor_at(fecha_val.day_diff(cshflw.get_settlement_date()))
    amt_der = [a * bp * df for a in amt_der]
    if len(amt_der) > 0:
        result.append(np.array(amt_der))
total = result[0] * 0

for r in result:
    total += r

for i in range(len(total)):
    print(f"Sensibilidad en {i}: {total[i]:0,.2f}")

print(f"Sensibilidad de proyección: {sum(total):,.2f} CLP")

Sensibilidad en 0: -0.00
Sensibilidad en 1: -0.00
Sensibilidad en 2: -0.00
Sensibilidad en 3: -0.00
Sensibilidad en 4: -0.00
Sensibilidad en 5: -0.00
Sensibilidad en 6: -0.00
Sensibilidad en 7: -0.00
Sensibilidad en 8: -0.00
Sensibilidad en 9: -0.00
Sensibilidad en 10: -0.00
Sensibilidad en 11: -0.00
Sensibilidad en 12: -0.00
Sensibilidad en 13: -0.00
Sensibilidad en 14: -0.00
Sensibilidad en 15: -0.70
Sensibilidad en 16: -0.00
Sensibilidad en 17: -1,997.46
Sensibilidad en 18: -0.00
Sensibilidad en 19: -0.00
Sensibilidad en 20: -0.00
Sensibilidad en 21: -0.00
Sensibilidad en 22: -0.00
Sensibilidad en 23: -0.00
Sensibilidad en 24: -0.00
Sensibilidad en 25: -0.00
Sensibilidad en 26: -0.00
Sensibilidad en 27: -0.00
Sensibilidad en 28: -0.00
Sensibilidad en 29: -0.00
Sensibilidad en 30: -0.00
Sensibilidad en 31: -0.00
Sensibilidad en 32: -0.00
Sensibilidad de proyección: -1,998.16 CLP


**Ejercicio:** Ambas sensibilidades se cancelan. ¿Porqué?